# This is EDA sheet for Amazon Bestselling (550) books 
                                        
                                                        by- GAURAV GAUTAM

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd


In [ ]:
data = pd.read_csv('../input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv')

## lets have a look at data format

In [ ]:
data.head()

## bit more info

In [ ]:
data.info()

In [ ]:
# getting the years of data
data['Year'].unique()


 Data is for year 2009-2019

In [ ]:
data['Author'].nunique()

In [ ]:
data['Name'].nunique()

Number of unique 'Authors' and 'Book names' are less than total entries so theres repitition for both

Theres can be record of same book for multiple years 

but still authors are less than tha no. unique books

there are author having multiple books in this datset 

In [ ]:
max_release = data.groupby(['Genre','Author']).count().sort_values(by = 'Name',ascending  = False).head(10)
max_release

In [ ]:
# lets see if these author have seperate releases or same book
# lets check for first only 
data[data['Author'] == 'Jeff Kinney']

 Here we are having different releases for this author , so there are multiple books from same author 
 it can be a light correlation with author name as this author has rating above 4.7 yet with small number of reviews 

## lets explore feature charecterstics individually

In [ ]:
# count of fictional and non-fictional books 
data.groupby('Genre').count()

In [ ]:
data['Price'].describe()

# Now User-rating and reviews are two most important factors for recommendation or predicting the rating
so lets work with them and find correlation with other features like price , genre , if any.

In [ ]:
data.groupby('Genre').describe()[['User Rating','Price','Reviews']].transpose()
# most of the corr of genre with other attributes can be drawn from here , a better picture will come from visualization later 
# """ Some Conclusions 
#     mean user rating for fiction is quite more even no. of fiction books are less means people like fiction more 
#     non- fiction books are quite costly than fiction , that could be reason for less readers-less ratings and reviews 

In [ ]:
data.sort_values(by ='User Rating',ascending = False)
# these are top rated books 
# lets get more precise order by sorting with no. of reviews afterward

In [ ]:
data.sort_values(by =['User Rating','Reviews'],ascending = [False,False]).head(10)
# it looks a major hit , wit record number of reviews fo 4.9 rating 
# so we have rating for similar book too for different years , thats dominating rating mean,
# i dont think so that every year new reviews were given to same book
# so its same data recurring , i think it should be taken once , advice
# lets see how many unique books are there

In [ ]:
data['Name'].nunique()
# out of 550 entries we have ony 351 unique books
# lets have dataframe of unique books

## New data set with only unique books

Lets drop repeated books and work with only unique ones 
and have the mean values for the repeated ones

Because i dont think just for different yeas to include the same data , and year is not much productive for prediction unless we do some more research related to that time .

In [ ]:
mean_data = data.groupby('Name').mean().sort_values(by=['User Rating','Reviews'],ascending = [False,False])
mean_data['Year'] = mean_data['Year'].apply(int)
mean_data['Price'] = mean_data['Price'].apply(int)
mean_data.reset_index(inplace = True)
mean_data
# taking only first entry of each book
# theres no meaning of having year as float 

In [ ]:
# lets see how  many books were repeated 
recur= data.groupby('Name').count()
recur.rename(columns={'Author':'Count'},inplace = True)
recur = recur['Count']
recur_i = recur.count()
recur_f = recur[recur>1].count()

In [ ]:
print(recur_i,recur_f)
# 91 books have repeatitions in 550 entries 
# 351 are unique

#  visualization
This gave just an idea about relations between various features and to understand the data ,

Now, lets do some visualization to get some more clear picture and analyze trends 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import cufflinks as cf

Genre column was missing in  groupby names , now including usinng merge, author can be included too but lets drop it  

In [ ]:
data

In [ ]:
mean_data = pd.merge(mean_data,data[['Name','Genre']].drop_duplicates(subset = 'Name'),on='Name',how='left')

In [ ]:
mean_data

In [ ]:
sns.pairplot(mean_data,hue = 'Genre')

Many rough ideas can be drawn from here about trends and also ideas to plot seperate plots for more precise observation

1) Year vs 

    use rating - no clear relation
    
    reviews - increase in reviews with time overall and for non-fiction as well( further plot),not clear surpassing fiction or not but for non-fiction itself, its getting more readers
    
    price - non-fiction seems costlier somewhere but no corr with year
    
    year - non-fiction surpassing fiction with time 
 
2) Price vs
    user rating - not significant
    reviews - non-fiction have less number of reviews and have higher price (although weak but price vs readers relation can be drawn)
    price -overall non-fiction quite costlier than fiction
    
3) Reviews - (a parameter to number of readers)
    user rating - overall non-fiction has less readers than fiction

4) User-rating -  non-fiction have higher kde plot - better overall rating as fiction has diverse range of rating from 3.2 to 5
    but non-fiction is mostly among 4-5
    

    

## year vs Reviews

In [ ]:
plt.figure(figsize = (15,10))
sns.boxplot(x = 'Year',y = 'Reviews',data = mean_data, hue = 'Genre')

Now , it can be clearly seen that from 2013-2015 fictional were at great demand 

Also,  Initially non-fiction had very less number of readers ,increasing from 2011 with small amount but significantly from 2016 ,  showing the shift of interest from fictional to non-fictional with passing time  

## Year vs Genre

In [ ]:
plt.figure(figsize = (15,6))
sns.displot(x='Year',hue='Genre',data = mean_data,palette='cool')

# Non-fiction showed a significant presence and increase among best sellers 2016 onwards
although increase can be seen from 2010,outlying fiction  but was not that much significant

2019 has extraordinary presence of non-fiction among best sellers 

In [ ]:
plt.figure(figsize=(15,5))
plt.tight_layout()
sns.stripplot(x = 'Price' , y = 'Reviews', data=mean_data,hue = 'Genre')

## most of fictional are between 4-20
but non fictional are much costlier between 4-40 ----
Fictional are more at min ----
and non-fiction is  at max ----
Fictional have greater number of reviews thus more readers ----
for price - reviews ----
books between 4-22 have most readers 

# Conclusion:
theres no significannt correlation between any of features - price , reviews , user-rating,year 
few weak correlations have been drawn
